In [1]:
!pip install scikit-learn

### Pandas for handling the data
import pandas as pd

## Modules for content-based filtering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import time


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Load the dataset and create dataframe variable
dfAnime = pd.read_csv('anime-dataset-2023.csv')
print("Shape of the Dataset:", dfAnime.shape)
dfAnime.head()

Shape of the Dataset: (24905, 24)


,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


In [3]:
# Inicializace TF-IDF vektorizeru
tfidf = TfidfVectorizer()

# Vytvoření tfidf matice pro získání důležitosti individuálních žánru ve vztahu k anime
tfidfMatrix = tfidf.fit_transform((genre for genre in dfAnime['Genres'].values.astype('U')))

# Výpočet matice kosinove podobnosti 
cosineSimMatrix = linear_kernel(tfidfMatrix, tfidfMatrix)

In [4]:
# Zobrazeni TF-IDF matice
print(tfidfMatrix)

# Zobrazení matice kosinové podobnosti
print(cosineSimMatrix)

# Ziskani features names z vektorizeru
genreNames = tfidf.get_feature_names_out()
genreId = 0

# Získání jména žánru
print("Genre name of id {}: ".format(str(genreId)))
print(genreNames[genreId])

# Získání jména anime
animeNames = dfAnime['Name'].tolist()

  (0, 10)	0.32602457103771887
  (0, 21)	0.32602457103771887
  (0, 27)	0.595147585199671
  (0, 3)	0.595147585199671
  (0, 0)	0.2810954674507281
  (1, 10)	0.6037505476446903
  (1, 21)	0.6037505476446903
  (1, 0)	0.5205483190228102
  (2, 1)	0.48940985399161224
  (2, 10)	0.5265030163465828
  (2, 21)	0.5265030163465828
  (2, 0)	0.4539461888504063
  (3, 24)	0.5346180163212002
  (3, 18)	0.6141171661851885
  (3, 6)	0.44479659748851075
  (3, 0)	0.3730947194535217
  (4, 9)	0.4707570469734566
  (4, 24)	0.7050002208806446
  (4, 1)	0.5304361330858535
  (5, 23)	1.0
  (6, 20)	0.6679764105607336
  (6, 5)	0.43039072015109003
  (6, 6)	0.6071007683591121
  (7, 16)	0.45268300778328474
  (7, 19)	0.45268300778328474
  :	:
  (24894, 4)	0.73759499435081
  (24894, 17)	0.6752433815363378
  (24895, 20)	0.8079025256636946
  (24895, 9)	0.5893161367434488
  (24896, 0)	1.0
  (24897, 6)	1.0
  (24898, 11)	0.7071067811865476
  (24898, 2)	0.7071067811865476
  (24899, 11)	0.7071067811865476
  (24899, 2)	0.707106781186547

In [5]:
# Funkce k získání výpisu určitého počtu anime na základě zadaného názvu, kosinové podobnosti a hodnocení z datasetu
def getRecommendations(title, cosineSimMatrix, dfAnime, count):
    idx = dfAnime[dfAnime['Name'] == title].index[0]

    # Nalezení podobnosti zadaného anime s ostatními anime v matici podobnosti
    simScores = list(enumerate(cosineSimMatrix[idx]))

    # Vyfiltrování anime s neznámým hodnocením
    validScores = [x for x in simScores if dfAnime.iloc[x[0]]['Score'] != "UNKNOWN"]

    # Seřazení podle podobnosti a hodnocení
    sortedScores = sorted(validScores, key=lambda x: (x[1], dfAnime.iloc[x[0]]['Score']), reverse=True)

    # Vyloučení zadaného anime z listu
    topAnimes = [x for x in sortedScores if x[0] != idx][:count]

    # Vytvoření listu
    recommendedIndices = [idx for idx, _ in topAnimes]
    recommendedAnimes = dfAnime.iloc[recommendedIndices][['Name', 'Genres', 'Score']]
    return recommendedAnimes

In [7]:
animeTitle = 'Mushishi'
recommendations = getRecommendations(animeTitle, cosineSimMatrix, dfAnime, 10)
print(f'Recommendations for "{animeTitle}":')
print(recommendations)

Recommendations for "Mushishi":
                                      Name  \
9151         Mushishi Zoku Shou 2nd Season   
8532                    Mushishi Zoku Shou   
9875   Mushishi Zoku Shou: Suzu no Shizuku   
8404                  Mushishi: Hihamukage   
9148    Mushishi Zoku Shou: Odoro no Michi   
2668                                 Mokke   
3544                        Mokke Specials   
6692                                Hyouka   
6966             Hyouka: Motsubeki Mono wa   
21585    Meitantei Conan: Zero no Tea Time   

                                                Genres Score  
9151   Adventure, Mystery, Slice of Life, Supernatural  8.73  
8532   Adventure, Mystery, Slice of Life, Supernatural   8.7  
9875   Adventure, Mystery, Slice of Life, Supernatural  8.59  
8404   Adventure, Mystery, Slice of Life, Supernatural  8.54  
9148   Adventure, Mystery, Slice of Life, Supernatural  8.43  
2668              Mystery, Slice of Life, Supernatural  7.05  
3544              My